# Squid Game Effect
<a id="toc"></a>
Table of Contents
* [1.0 Introduction & Methodology](#section_01)
* [2.0 Data Sources & ETL Process](#section_02)
* [3.0 Part 1 The Mystery - Unexpected Global Interest](#section_03)
* [4.0 Part 2 The Correlation Evidence](#section_04)
* [5.0 Part 3 The Smoking Gun - Squid Game Experiments](#section_05)
* [6.0 Part 4 The Geographic Proof](#section_06)
* [7.0 Part 5 The Sustained Wave - Beyond Viral Moments](#section_07)
* [8.0 Limitations & Conclusions](#section_08)
* [9.0 Credits](#section_09)

--------------------------------

<a id="section_01"></a>
## 1.0 Introduction
For decades, language learning was predictable: English, Spanish and French were the languages of choice. Hoowever, in 2018, Duolingo's data detectives spotted an anomaly. They noticed that interests in Korean culture and language are surging in countries with no historical ties to Korea.

Their hyoothesis? The K-pop and K-drama effect.  Duolinga began weaving famous K-drama lines into lessons. Their latest campaign of "Learn Korean or Else" was a partnership with Netflix in late 2024 with the release of "Squid Game Season 2".

This project aims to start on a data trial to see if this is just a corporate intuition based on fads or that binge-watching K-drama is indeed becoming the world's newest Korean classrooms.

###### [↩️ Back to Table of Contents](#toc)

------------------------

<a id="section_02"></a>
## 2.0 Data Sources & ETL Process


### 2.1 Duolingo 2024 Language Report
* [Duolingo 2024 Report](https://docs.google.com/spreadsheets/d/1CndYC5ZovYfmPuMN9T9Jxfa4CQXOzZrfQ2kAUaWG1ZU/edit?ref=blog.duolingo.com&gid=532174835#gid=532174835
)
* [Google Docs](https://docs.google.com/spreadsheets/d/1CndYC5ZovYfmPuMN9T9Jxfa4CQXOzZrfQ2kAUaWG1ZU/edit?ref=blog.duolingo.com&gid=532174835#gid=532174835)

### 2.2 Google Trends 5-year Time Lines & Regional Data
* _Data Collected on 1 Nov. 2025_

### 2.3 Google Trends Timeline & Region Data For Squid Game 1 & Squid Game 2
* 1 year period covering 9 months before the release of each show and 3 months after
    * Squid Game 1 was released on Netflix on 17 September 2021
    * Squid Game 2 was released on Netflix on 26 December 2024
* _Data Collected on 1 Nov. 2025_


### 2.4 Netflix Movies & TC shows Dataset
* source: [HQ DATA PROFILER on Kaggle](https://www.kaggle.com/datasets/hqdataprofiler/cleaned-netflix-movies-and-tv-shows)

### 2.5 My Drama List
* source: [REDHATA on Kaggle](https://www.kaggle.com/datasets/redhata/korean-drama-dataset-2010-2025-2600-titles)

###### [↩️ Back to Table of Contents](#toc)

--------------------------------

<a id="section_03"></a>
## 3.0 Part 1 The Mystery - Unexpected Global Interest

###### [↩️ Back to Table of Contents](#toc)

--------------------------------

<a id="section_04"></a>
## 4.0 Part 2 The Correlation Evidence

###### [↩️ Back to Table of Contents](#toc)

--------------------------------

<a id="section_05"></a>
## 5.0 Part 3 The Smoking Gun - Squid Game Experiments

###### [↩️ Back to Table of Contents](#toc)

--------------------------------

<a id="section_06"></a>
## 6.0 Part 4 The Geographic Proof

###### [↩️ Back to Table of Contents](#toc)

--------------------------------

<a id="section_07"></a>
## 7.0 Part 5 The Sustained Wave - Beyond Viral Moments

###### [↩️ Back to Table of Contents](#toc)

--------------------------------

<a id="section_08"></a>
## 8.0 Limitations & Conclusions

###### [↩️ Back to Table of Contents](#toc)

--------------------------------

<a id="section_09"></a>
## 9.0 Credit

###### [↩️ Back to Table of Contents](#toc)

--------------------------------